In [ ]:
import pandas as pd
from prophet import Prophet
Policies = pd.read_csv('../data/01_raw_data/policies.csv')

Policies.columns

Index(['PolicyID', 'ClientID', 'AgentID', 'PolicyType', 'StartDate', 'EndDate',
       'PremiumAmount', 'Status', 'Channel', 'PaymentStatus', 'RiskScore',
       'OriginalPolicyID', 'RenewalNumber', 'BusinessType', 'TransactionType',
       'CoverageType', 'VehicleAge', 'UsageType'],
      dtype='object')

In [16]:
# Ensure correct column names and types
Policies['StartDate'] = pd.to_datetime(Policies['StartDate'], errors='coerce')

# Now group by month using the correct column
monthly_sales = Policies.groupby(pd.Grouper(key='StartDate', freq='M'))['PremiumAmount'].sum().reset_index()

monthly_sales.rename(columns={'StartDate':'ds','PremiumAmount':'y'}, inplace = True)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12508\2242330630.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = Policies.groupby(pd.Grouper(key='StartDate', freq='M'))['PremiumAmount'].sum().reset_index()


In [17]:
monthly_sales.head()

,ds,y
0,2022-09-30,1735948.27
1,2022-10-31,2365865.93
2,2022-11-30,3577077.38
3,2022-12-31,6427020.88
4,2023-01-31,8524065.72


In [19]:
from prophet import Prophet

model = Prophet()
model.fit(monthly_sales)

future = model.make_future_dataframe(periods=6,freq = 'M')
forecast = model.predict(future)

15:48:57 - cmdstanpy - INFO - Chain [1] start processing
15:48:57 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\ADMIN\anaconda3\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [20]:
forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df.rename(columns={'ds': 'Date', 'yhat': 'Predicted', 'yhat_lower': 'LowerBound', 'yhat_upper': 'UpperBound'}, inplace=True)

# Merge actual + forecast for comparison
final_df = pd.merge(monthly_sales, forecast_df, how='right', left_on='ds', right_on='Date')


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12508\348561969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_df.rename(columns={'ds': 'Date', 'yhat': 'Predicted', 'yhat_lower': 'LowerBound', 'yhat_upper': 'UpperBound'}, inplace=True)


In [21]:
final_df.to_csv('../data/01_raw_data/sales_forecast.csv', index=False)


In [22]:
final_df.head(10)

,ds,y,Date,Predicted,LowerBound,UpperBound
0,2022-09-30,1735948.27,2022-09-30,6.406587e+06,2.214379e+06,1.033329e+07
1,2022-10-31,2365865.93,2022-10-31,5.084311e+06,8.520178e+05,9.196452e+06
2,2022-11-30,3577077.38,2022-11-30,6.657082e+06,2.772781e+06,1.030141e+07
3,2022-12-31,6427020.88,2022-12-31,8.625901e+06,4.448176e+06,1.298511e+07
4,2023-01-31,8524065.72,2023-01-31,9.291405e+06,5.304497e+06,1.323466e+07
5,2023-02-28,9890862.03,2023-02-28,1.176642e+07,7.712871e+06,1.564567e+07
6,2023-03-31,11129586.94,2023-03-31,1.348215e+07,9.153409e+06,1.760999e+07
7,2023-04-30,10879458.26,2023-04-30,1.012179e+07,6.138833e+06,1.421613e+07
8,2023-05-31,12863903.11,2023-05-31,1.244639e+07,8.335900e+06,1.646656e+07
9,2023-06-30,14246531.80,2023-06-30,1.275124e+07,8.713596e+06,1.702475e+07
